In [1]:
# ============================================================
#  Fibonacci Word → Three Evolving Transforms
#  ------------------------------------------------------------
#  • Raw convergent B_n  (fractional value 0.xxx)
#  • Odds–ratio          R_n = B_n / (1 - B_n)
#  • Tail–difference     T_n = 1 / (B_{n+1} - B_n)
#  • Binary mod inverse  M_n = m_n / 2^{F_n}  with  m_n * num ≡ 1 (mod 2^{F_n})
#
#  Paste this *single* cell into Colab and run.  No user input needed.
#  It prints a tidy table of parts‑per‑million (ppm) gaps to α⁻¹, π, e, √2
#  for each transform and each Fibonacci index n = 15..18.
# -------------------------------------------------------------
import mpmath as mp
mp.mp.dps = 100  # high precision across all transforms

# ---------- helpers -----------------------------------------
# Fibonacci numbers F[0]=0 so that F[15]=610, F[18]=2584 ...
F = [0, 1]
for _ in range(2, 25):
    F.append(F[-1] + F[-2])

def fib_word_bits(n_bits: int) -> str:
    """Return exactly n_bits of the (forward) Fibonacci word."""
    a, b = '0', '01'
    w = '0'
    while len(w) < n_bits:
        w = ''.join(b if ch == '0' else a for ch in w)
    return w[:n_bits]

def B_convergent(n_idx: int) -> mp.mpf:
    """Fractional value 0.xxx from the first F[n_idx] bits."""
    bits = fib_word_bits(F[n_idx])
    num  = int(bits, 2)
    den  = 1 << F[n_idx]      # 2**F_n (fast)
    return mp.mpf(num) / den, num, den  # also return raw numerator/denominator

# ---------- targets -----------------------------------------
alpha_inv = mp.mpf('137.035999146')  # CODATA‑2022
TARGETS = {
    'alpha' : alpha_inv,
    'pi'    : mp.pi,
    'e'     : mp.e,
    'sqrt2' : mp.sqrt(2),
}

# ---------- table header ------------------------------------
print("{:<6} {:>8} {:>10} {:>14} {:>14} {:>14}".format(
    "n", "F_n", "transform", "α ppm", "π ppm", "e ppm", "√2 ppm"))
print("-"*70)

# we iterate through n = 15..18  (have B_{n+1} ready for tail‑diff)
for n in range(15, 19):
    Bn, num, den = B_convergent(n)
    # Next convergent for T_n; safe because we pre‑generated F up to 24
    Bnext, _, _ = B_convergent(n+1)

    # ---- TRANSFORM 1: raw B_n ----------------------------------
    transforms = {
        'B_n': Bn,
        'R_n': Bn / (1 - Bn),
        'T_n': mp.mpf('inf') if Bnext == Bn else 1 / (Bnext - Bn),
    }

    # ---- TRANSFORM 3: binary modular inverse -------------------
    # Requires numerator odd to have inverse mod 2^k.  If even, skip.
    if num % 2 == 1:
        m_inv = pow(num, -1, den)   # Python ≥3.8  modular inverse
        Mn = mp.mpf(m_inv) / den
        transforms['M_n'] = Mn
    else:
        transforms['M_n'] = None  # mark unattainable at this n

    # ---- print ppm gaps ----------------------------------------
    for label, value in transforms.items():
        if value is None:
            # no inverse exists
            print(f"{n:<6} {F[n]:>8} {label:>10}    {'N/A':>10} {'N/A':>14} {'N/A':>14} {'N/A':>14}")
            continue
        gaps = [abs(value - tgt) / tgt * 1e6 for tgt in TARGETS.values()]
        print("{:<6} {:>8} {:>10} {:>14} {:>14} {:>14} {:>14}".format(
            n, F[n], label,
            mp.nstr(gaps[0], 6),
            mp.nstr(gaps[1], 6),
            mp.nstr(gaps[2], 6),
            mp.nstr(gaps[3], 6)))
    print("-")

# -------------------------------------------------------------
print("Finished.  Inspect ppm trends: only a falling α curve signals a promising transform.")


# ░░░  EXTRA CONSTANT SUITE  ░░░
#   – sqrt(π), sqrt(e), sqrt(√2)
#   – 1 – each of the above
#   – reciprocals of those six
#   –   π/e/√2 divided by 3 and by √3
#   – 1 – each of those four
#   – reciprocals of those four
#
# All printed as ppm gaps versus B_n (add your R_n, T_n, M_n
# if you want – the loop structure is identical).

import mpmath as mp
mp.mp.dps = 80

# --- BASIC CONSTANTS -------------------------------------------------------
pi      = mp.pi
econst  = mp.e
rt2     = mp.sqrt(2)
rt3     = mp.sqrt(3)

consts  = {
    'sqrt_pi' : mp.sqrt(pi),
    'sqrt_e'  : mp.sqrt(econst),
    'sqrt_rt2': mp.sqrt(rt2),

    '1-m√'    : 1 - mp.sqrt(pi),          # will rename below for clarity
    '1-me√'   : 1 - mp.sqrt(econst),
    '1-mr√2'  : 1 - mp.sqrt(rt2),

    'inv√π'   : 1/mp.sqrt(pi),
    'inv√e'   : 1/mp.sqrt(econst),
    'inv√√2'  : 1/mp.sqrt(rt2),

    # π/e/√2 divided by 3 or √3
    'π/3'     : pi/3,
    'e/3'     : econst/3,
    '√2/3'    : rt2/3,
    'π/√3'    : pi/rt3,
    'e/√3'    : econst/rt3,
    '√2/√3'   : rt2/rt3,

    # 1 – those
    '1-π/3'   : 1 - pi/3,
    '1-e/3'   : 1 - econst/3,
    '1-√2/3'  : 1 - rt2/3,
    '1-π/√3'  : 1 - pi/rt3,
    '1-e/√3'  : 1 - econst/rt3,
    '1-√2/√3' : 1 - rt2/rt3,
}

# reciprocals of the last four groups
extra = {}
for k, v in list(consts.items()):
    extra[f'inv({k})'] = 1/v
consts.update(extra)

# rename the earlier ambiguous keys for readability
consts = {k.replace('1-m√', '1-√π')
            .replace('1-me√', '1-√e')
            .replace('1-mr√2', '1-√√2') : v
          for k, v in consts.items()}

# --- your B_convergent helper must exist -------------------------------
# assume F array & fib_word_bits already defined earlier
def B_convergent(n_index: int) -> mp.mpf:
    bits = fib_word_bits(F[n_index])
    return mp.mpf(int(bits, 2)) / (1 << F[n_index])

# ----------------------------------------------------------------------
def ppm_gap(x, y):
    return abs(x - y) / y * 1e6

for n in [15, 16, 17, 18]:
    Bn = B_convergent(n)
    print(f'\n-- n={n}  F_n={F[n]}  B_n={mp.nstr(Bn, 8)} --')
    for name, val in consts.items():
        gap = ppm_gap(Bn, val)
        if gap < mp.mpf('1e-3'):
            # highlight tiny gaps explicitly
            print(f'  {name:<10}  {mp.nstr(gap,8):>12} ppm  ***')
        else:
            print(f'  {name:<10}  {mp.nstr(gap,8):>12} ppm')



n           F_n  transform          α ppm          π ppm          e ppm
----------------------------------------------------------------------
15          610        B_n       997882.0       907628.0       893243.0       794800.0
15          610        R_n       997017.0       869862.0       849596.0       710906.0
15          610        T_n           +inf           +inf           +inf           +inf
15          610        M_n       996273.0       837412.0       812093.0       638821.0
-
16          987        B_n       997882.0       907628.0       893243.0       794800.0
16          987        R_n       997017.0       869862.0       849596.0       710906.0
16          987        T_n           +inf           +inf           +inf           +inf
16          987        M_n           N/A            N/A            N/A            N/A
-
17         1597        B_n       997882.0       907628.0       893243.0       794800.0
17         1597        R_n       997017.0       869862.0       849596.0